In [ ]:
from hanlp_restful import HanLPClient
import json
import os
import time

HanLP = HanLPClient('https://www.hanlp.com/api', auth='', language='zh')
data=HanLP.parse('中央亞非利加之蠻地探險英國大探險家李秉鐸司徒雷之實地探險談王桐齡巍然聳於沙上者金字塔崢嶸究兀高舉怪異之頭者獅身人面像澎湃洶湧發於古壹之哀音者尼羅河之長流赤帽黑衣悠悠高然據於駝背上者土人之怪狀亞非利加東北一隅之埃及自旅行者觀之已屬一種不可思議之國', 
            tasks='ner/ontonotes')

In [2]:

tags=data['ner/ontonotes']
print(tags)


[[['亞非利加', 'LOC', 1, 2], ['英國', 'NORP', 6, 7], ['李秉鐸', 'PERSON', 9, 10], ['司徒雷', 'PERSON', 10, 11], ['王桐齡', 'PERSON', 15, 16], ['尼羅河', 'LOC', 43, 44], ['亞非利加', 'LOC', 62, 63], ['埃及', 'GPE', 66, 67]]]


In [3]:
def split_into_blocks(text, segment_size=150):
    # split text into a list of segments
    segment_list = []
    i = -1
    while True:
        i += 1
        left, right = i*segment_size, (i+1)*segment_size
        segment = text[left: right]
        segment_list.append(segment)
        if right >= len(text):
            break
    return segment_list

In [4]:
block_size = 150
big_block_size = 14000
results_dict = dict()

In [5]:
def join(ll):
    # merge a list of lists
    return_l = []
    for l in ll:
        return_l = return_l + l
    return return_l

In [ ]:
results_dict = dict()

In [6]:
texts = []
query_timestamps = []
root = "trans"
for file_name in os.listdir(root):
    print("currently processing", file_name)
    if file_name in results_dict:
        continue
    with open(os.path.join(root, file_name), 'r', encoding='utf-8') as rf:
        text = rf.read()
#         def read_in_blocks(text, block_size=15000):
#             while True:
#                 block = text.read(block_size)
#             if not block:
#                 return  # 文件读取完毕
#         read_in_blocks(text)
    
    all_tokens = []
    all_tags = []
    
    big_block_list = split_into_blocks(text, 14000)
    
    for big_block in big_block_list:
        block_list = split_into_blocks(big_block, 150)

        if (len(query_timestamps) < 60):
            gap = 10000
        else:
            current_time = time.time()
            gap = current_time - query_timestamps[-60]
        if gap < 70:
            print("\tsleeping for", 70-gap)
            time.sleep(70-gap)
        query_timestamps.append(time.time())
        
        # 分词
        data = HanLP.parse(block_list, tasks='ner/ontonotes')
        tokens = data['tok/fine']
        tags = data['ner/ontonotes']

        all_tokens = all_tokens + join(tokens)
        all_tags = all_tags + join(tags)

    results_dict[file_name] = {"all_tokens": all_tokens, "all_tags": all_tags}
#     print (all_tokens)

currently processing 一落千丈.txt
currently processing 七星宝石.txt
currently processing 七次冒险记.txt
currently processing 三千年艳尸记.txt
currently processing 世界一周.txt
currently processing 中央亚细亚探险记.txt
currently processing 二勇少年.txt
currently processing 二十六点钟大飞行.txt
currently processing 二千磅.txt
currently processing 五大洲探险记.txt
currently processing 侠女郎.txt
currently processing 侠隐记.txt
currently processing 倭利物冒险记.txt
currently processing 八十日.txt
currently processing 八十日环游记.txt
currently processing 冰原探险记.txt
currently processing 冰天漁樂記.txt
currently processing 冰山雪海.txt
currently processing 冰洋双鲤.txt
currently processing 冰窟余生录.txt
currently processing 别而治盗记.txt
currently processing 动物院叟.txt
currently processing 劫里慈航.txt
currently processing 勇敢童子.txt
currently processing 北京巴黎間競車日記.txt
currently processing 北冰洋探険録.txt
currently processing 北极出险记.txt
currently processing 十五小豪杰.txt
currently processing 双雄较剑录.txt
currently processing 双鸳脱罗记.txt
currently processing 古国幽情记.txt
currently processing 古鬼遺金記.txt
currently 

In [7]:
import json

with open("results_dict_ner_tag_trans.json", "w", encoding='utf-8') as f:
    json.dump(results_dict, f)



In [ ]:
# with open("results_dict_ner_tag_trans.json", "r", encoding='utf-8') as f:
#      results_dict = json.load(f)

In [14]:
tags_we_care = {'LOC', 'NORP', 'GPE'}
for file_name in results_dict:
    all_tokens, all_tags = results_dict[file_name]['all_tokens'], results_dict[file_name]['all_tags']
    
    count_dict = dict()
    for [phrase, tag, _, _] in all_tags:
        if tag in tags_we_care:
            key = (phrase, tag)
            if key not in count_dict:
                count_dict[key] = 1
            else:
                count_dict[key] += 1
    results_dict[file_name]['tag_freq'] = count_dict

In [9]:
def merge_dict(dict_list):
    return_dict = dict()
    for d in dict_list:
        # merge d into return_dict
        for key, val in d.items():
            if key not in return_dict:
                return_dict[key] = 0
            return_dict[key] += val
    return return_dict

In [10]:
all_tag_freq = merge_dict([results_dict[file_name]['tag_freq'] for file_name in results_dict])

In [11]:
l = [(key, val) for (key, val) in all_tag_freq.items()]
l.sort(key=lambda var: var[1], reverse=True)
l

[(('英國', 'GPE'), 827),
 (('英', 'GPE'), 748),
 (('英國', 'NORP'), 715),
 (('倫敦', 'GPE'), 611),
 (('巴黎', 'GPE'), 588),
 (('法國', 'GPE'), 522),
 (('俄', 'GPE'), 516),
 (('月球', 'LOC'), 462),
 (('西班牙人', 'NORP'), 426),
 (('印度', 'GPE'), 410),
 (('西班牙', 'GPE'), 391),
 (('埃及', 'GPE'), 387),
 (('英', 'NORP'), 377),
 (('地球', 'LOC'), 370),
 (('歐洲', 'LOC'), 360),
 (('非洲', 'LOC'), 334),
 (('即墨', 'GPE'), 260),
 (('西班牙', 'NORP'), 258),
 (('日', 'GPE'), 219),
 (('荷蘭', 'GPE'), 217),
 (('日本', 'GPE'), 215),
 (('法國', 'NORP'), 213),
 (('埃及', 'NORP'), 207),
 (('歐', 'LOC'), 199),
 (('印度人', 'NORP'), 194),
 (('中國', 'GPE'), 185),
 (('德', 'GPE'), 185),
 (('蘇嚕', 'NORP'), 183),
 (('大陸', 'GPE'), 180),
 (('俄國', 'GPE'), 179),
 (('美洲', 'LOC'), 168),
 (('法', 'GPE'), 167),
 (('紐約', 'GPE'), 162),
 (('開羅', 'GPE'), 159),
 (('美國', 'GPE'), 158),
 (('香港', 'GPE'), 154),
 (('猶太人', 'NORP'), 153),
 (('羅馬', 'GPE'), 151),
 (('北極', 'LOC'), 151),
 (('大西洋', 'LOC'), 146),
 (('孟買', 'GPE'), 141),
 (('太平洋', 'LOC'), 140),
 (('美國', 'NORP'), 133),


In [12]:
import pandas as pd
df = pd.DataFrame([[a,b,c] for ((a, b), c) in l], columns=['phrase', 'tag', 'frequency'])
print(df)

      phrase   tag  frequency
0         英國   GPE        827
1          英   GPE        748
2         英國  NORP        715
3         倫敦   GPE        611
4         巴黎   GPE        588
...      ...   ...        ...
11503     藏門   LOC          1
11504     赤帝   LOC          1
11505     嘉蜒   LOC          1
11506     司城   LOC          1
11507    逍德堡   LOC          1

[11508 rows x 3 columns]


In [13]:
df.to_excel('all_freq_trans.xlsx', sheet_name='new_sheet_name')

In [ ]:
results_dict['黄金毒.txt']['tag_freq']